In [54]:
import sys
import numpy as np
import pandas as pd
import requests

def get_input(user_id):
    adress = 'http://35.180.178.243'

    req = requests.get(adress + '/reset?user_id=' + user_id)

    data = req.json()

    nb_users = data.pop('nb_users')
    nb_items = data.pop('nb_items')
    next_state = data.pop('next_state')
    columns = ['state_history', 'action_history', 'rewards_history']
    data_array = np.array([data.pop(c) for c in columns])
    df =  pd.DataFrame(data_array.T, columns=columns)
    return nb_users, nb_items, next_state, data_array

In [55]:
ID = 'VI2X71V0287S9F9B7SCU'

In [78]:
nb_users, nb_items, next_state , data = get_input(ID)

In [119]:
state_history, action_history, reward_history = data
columns_names = ['user_id', 'item_id', 'item_price'] + ['metadata_' + str(i) for i in range(5)]
dfs = [pd.DataFrame(state_history[i], columns=columns_names) for i in range(len(state_history))]
df = pd.DataFrame(state_history[0], columns=columns_names)
df['hist_line'] = 0
for i in range(1, len(state_history)):
    df_temp = pd.DataFrame(state_history[i], columns=columns_names)
    df_temp['hist_line'] = i
    df = df.append(df_temp, ignore_index=True)
    
df_next_state = pd.DataFrame(next_state, columns=columns_names)

In [85]:
print('Nombre d\'utilisateurs :', nb_users)
print('Nombre d\'items :', nb_items)

Nombre d'utilisateurs : 100
Nombre d'items : 30


## Utilisateurs/items non présents dans l'historique

In [107]:
print('Nombre d\'utilisateurs non présents', nb_users-df['user_id'].nunique())
print('Nombre d\'items non présents', nb_items-df['item_id'].nunique())

Nombre d'utilisateurs non présents 17
Nombre d'items non présents 0


## Metadonnées utilisateur

In [108]:
for i in range(5):
    print('Metadata', i, ': ', (df.groupby('user_id')['metadata_' + str(i)].nunique() > 1).sum() == 0)

Metadata 0 :  True
Metadata 1 :  True
Metadata 2 :  False
Metadata 3 :  False
Metadata 4 :  False


Les metadonnées 0 et 1 sont des métadonnées utilisateurs (d'après l'historique)

## Metadonnées produits

In [109]:
for i in range(5):
    print('Metadata', i, ': ', (df.groupby('item_id')['metadata_' + str(i)].nunique() > 1).sum() == 0)

Metadata 0 :  False
Metadata 1 :  False
Metadata 2 :  True
Metadata 3 :  True
Metadata 4 :  False


Les metadonnées 2 et 3 sont des métadonnées utilisateurs (d'après l'historique)

## Nombre de commandes par utilisateurs

In [114]:
print('Nombre de commande par utilisateur minimum :', df.groupby('user_id')[['hist_line']].count().min()[0]) # 0 si on compte les absents
print('Nombre de commande par utilisateur maximum :', df.groupby('user_id')[['hist_line']].count().max()[0])

Nombre de commande par utilisateur minimum : 30
Nombre de commande par utilisateur maximum : 177


## Diversité des achats (quantitative)

In [118]:
print('Nombre d\'articles par utilisateur minimum :', df.groupby('user_id')[['item_id']].nunique().min()[0]) # 0 si on compte les absents

Nombre d'articles par utilisateur minimum :          item_id
user_id         
0             30
1             30
2             30
3             30
4             30
6             30
7             30
8             30
9             30
12            30
13            30
15            30
16            30
17            30
18            30
20            30
21            30
22            30
23            30
24            30
25            30
26            30
28            30
29            30
31            30
33            30
35            30
36            30
37            30
38            30
...          ...
67            30
68            30
69            30
71            30
72            30
73            30
74            30
75            30
76            30
77            30
78            30
79            30
80            30
82            30
83            30
84            30
85            30
86            30
87            30
88            30
89            30
90            30
91            30
92  

## Rapport au prix

In [83]:
#df[['user_id', 'item_price', 'metadata_2', 'metadata_3', 'metadata_4']].groupby('user_id').describe()
print('Minimum de la dépense moyenne :', df[['user_id', 'item_price']].groupby('user_id').mean().min()[0])
print('Maximum de la dépense moyenne :', df[['user_id', 'item_price']].groupby('user_id').mean().max()[0])
print('Minimum de l\'ecart type :', df[['user_id', 'item_price']].groupby('user_id').std().min()[0])
print('Maximum de l\'ecart type :', df[['user_id', 'item_price']].groupby('user_id').std().max()[0])


Minimum de la dépense moyenne : 487.4109794268577
Maximum de la dépense moyenne : 505.01067632104525
Minimum de l'ecart type : 282.1257686423793
Maximum de l'ecart type : 288.7819723945066


## Apparition de nouveaux utilisateurs ?

In [86]:
def send_pred(user_id, pred):
    adress = 'http://35.180.178.243'
    req = requests.get(adress + '/predict?user_id=' + user_id + '&recommended_item=' + str(pred))

    data = req.json()
    reward = data.pop('reward')
    state = data.pop('state')
    return reward, state

In [ ]:
no_new_user = True
while no_new_user:
    reward, state = send_pred(ID, 0)
    user_id = state[0][0]
    no_new_user = 

83